In [1]:
import numpy as np
import rasterio
import tensorflow as tf

In [2]:
block_size = 50

In [3]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)

In [4]:
def load_coords_file(fname: str):
    with open(fname, "r") as f:
        lines = f.readlines()
    return [tuple([float(v) for v in l.strip().split(",")]) for l in lines]

In [26]:
labels = ["boat", "land", "water", "windmill"]
indices = {"boat": 0, "land": 1, "water": 2, "windmill": 3}

wm_x_test = []
wm_y_test = []
wm_x_train = []
wm_y_train = []

for label in labels:
    training_coords = load_coords_file("././training/training_{}.txt".format(label))

    train_n = len(training_coords) / 5 * 4
    i = 0
    for coord in training_coords:
        x, y = dataset.index(coord[0], coord[1])
        img = img_data[x - block_size // 2:x + block_size // 2, y - block_size // 2:y + block_size // 2]
        if i < train_n:
            wm_x_train.append(img)
            wm_y_train.append(indices[label])
        else:
            wm_x_test.append(img)
            wm_y_test.append(indices[label])
        i += 1

wm_x_test = np.stack(wm_x_test)
wm_y_test = np.array(wm_y_test)
wm_x_train = np.stack(wm_x_train)
wm_y_train = np.array(wm_y_train)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(50, 50)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [36]:
model.fit(wm_x_train, wm_y_train, epochs=5)

2021-11-04 16:03:01.924962: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
3/3 [==============================] - 0s 4ms/step - loss: 711.9312 - accuracy: 0.1739
Epoch 2/5
3/3 [==============================] - 0s 3ms/step - loss: 325.0903 - accuracy: 0.3261
Epoch 3/5
3/3 [==============================] - 0s 3ms/step - loss: 198.8345 - accuracy: 0.3478
Epoch 4/5
3/3 [==============================] - 0s 3ms/step - loss: 179.8655 - accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 0s 3ms/step - loss: 93.5711 - accuracy: 0.4457


In [37]:
model.evaluate(wm_x_test,  wm_y_test, verbose=2)

1/1 - 0s - loss: 121.6552 - accuracy: 0.4783


[121.65522766113281, 0.47826087474823]